<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/22-save_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save and Load Model
PyTorch는 python의 pickle을 활용하여 직렬화(Serialize)하여 객체를 저장하고 역직렬화(Deserialize)를 통해 볼러올 수 있음.

In [ ]:
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

### `torch.save` & `torch.load`
- 모델의 정보를 경로에 저장하고 불러옴.
- 모델 전체를 저장하거나 상태만 저장할 때 모두 사용됨.
- `map_location`을 활용하면 device에 상관없이 불러올 수 있음.

## 비교
### Save
- 전체: `torch.save(model, 'model.pt')`
- 상태: `torch.save(model.state_dict(), model_params.pt')`

### load
- 전체: `torch.load('model.pt')`
- 상태: `model.load_state_dict(torch.load('model_params.pt'))`

In [ ]:
class Model(nn.Module):
  def __init__(self, n_in_f, n_out_f, w, b):
    super().__init__()
    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    init.constant_(self.l0.weight, w)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, b)

  def forward(self, x):
    return self.l0(x)

## 모델의 Parameters 저장
`state_dict`를 활용해 모델의 상태를 저장함.
- Structure 등은 저장되지 않음.
- PyTorch 버전에 관계없이 사용 가능.
- 모델을 불러올 때 동일한 형태의 클래스가 선언되어 있어야 함.

### `state_dict()`
- 객체의 상태를 가지고 있어 상태를 저장하고 복원할 수 있음.
- 모델의 parameter 상태와 buffer의 상태 저장
- 현재 Module instance의 상태에 해당하는 OrderedDict 객체 `state_dict` 반환.


### `load_state_dict(state_dict)`
현재 Module 인스턴스의 상태를 argument로 넘겨진 OrderedDict 객체 state_dict를 이용해 설정

- missing_keys : 로드하려는 state_dict에는 있으나 load_state_dict메서드를 호출한 Module 객체에는 없는 키들.
- unexpected_keys : Module 객체에는 있으나 인자로 넘겨진 state_dict에는 없는 키들.

### `torch.save(state_dict,'file_path') & state_dict=torch.load('file_path')`

`state_dict`객체는 torch의 `save` `load`를 이용해 파일로 저장되거나 로딩됨.

In [ ]:
# 모델 객체를 생성하고, 이에 대한 파라메터 확인후
# 파라메터만 저장.
model1 = Model(3, 1, 1., 0.5)
print(list(model1.named_parameters()))
torch.save(model1.state_dict(), 'model_params.pth')

# 새로운 모델 객체를 생성.
# 해당 모델 객체는 구조는 같으나, 파라메터들의 초기값은 다름.
model2 = Model(3, 1, 2., 1.5)

print('===============')
for old, new in zip(model1.parameters(), model2.parameters()):

  if not torch.equal(old,new):
    print('model and n_model w/ default init do not have parameters with the same values!')
    break
else:
  print('model and n_model w/ default init have parameters with the same values!')
print('===============')

# 이전 저장한 parameters에 대한 state_dict를
# 로드하고 해당 state_dict로 새로만든 모델의
# 파라메터를 설정하고 이전 모델과 비교.
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')
print(f'{type(loaded_params_ordered_dict)=}') # collections.OredredDict

ret_v = model2.load_state_dict(loaded_params_ordered_dict)
print(f'{type(ret_v)}: {ret_v}')

print('===============')
for old, new in zip(model1.parameters(), model2.parameters()):

  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]
model and n_model w/ default init do not have parameters with the same values!
type(loaded_params_ordered_dict)=<class 'collections.OrderedDict'>
<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
model and n_model have parameters with the same values!



## model 전체 저장
- Parameters와 Structure 함께 저장
- pickle에 의존하여 인스턴스를 직렬화하므로 모델 클래스 정의가 필요함.
- 버전이 크게 바뀔 경우 안 될 확률이 매우 높으므로 주기적으로 최신 버전으로 다시 저장 필요.

In [ ]:
# 저장할 모델 생성.
model = Model(3, 1, 2., 1.5)

# 모델 저장
torch.save(model, 'model.pth')

# 저장된 model 로드.
n_model = torch.load('model.pth')
print(f'{type(n_model)=}, {n_model}')

# 두 모델의 parameters비교.
for old, new in zip(model.parameters(), n_model.parameters()):
  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

type(n_model)=<class '__main__.Model'>, Model(
  (l0): Linear(in_features=3, out_features=1, bias=True)
)
model and n_model have parameters with the same values!
